In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [2]:
url = "https://app.metricwire.com/"
my_email = '***'
my_passw = '***'

In [45]:
date = '02-11-2020'

In [24]:
path_SBDL = "#root > div > main > div > div > div.MuiGrid-root.MuiGrid-item.MuiGrid-grid-xs-12.MuiGrid-grid-sm-12.MuiGrid-grid-md-8 > div > div.MuiCardContent-root > table > tbody > tr:nth-child(2) > td:nth-child(3) > a > span.MuiButton-label"
path_EDL = "#root > div > main > div > div > div.MuiGrid-root.MuiGrid-item.MuiGrid-grid-xs-12.MuiGrid-grid-sm-12.MuiGrid-grid-md-8 > div > div.MuiCardContent-root > table > tbody > tr:nth-child(1) > td:nth-child(3) > a > span.MuiButton-label"

pgconst_SBDL = 4
pgconst_EDL = 4


In [25]:
def loginMetricWire(my_email = my_email, my_passw = my_passw, OS = 'Mac'):
    '''
    INPUT: str formatted username and password for MetricWire account
    
    OUTPUT: driver for next function, simply opens in a test browser
    '''
    driver = webdriver.Chrome()
    
    
    driver.get(url)

    time.sleep(3)
    email_inp = driver.find_element_by_css_selector("#root > div > div > div > div > div > form > div:nth-child(4) > div > input")
    email_inp.send_keys(my_email)

    pass_inp = driver.find_element_by_css_selector("#root > div > div > div > div > div > form > div:nth-child(5) > div > input")
    pass_inp.send_keys(my_passw)

    login_btn = driver.find_element_by_css_selector("#root > div > div > div > div > div > form > div.card-action.border-0.text-right.mt-3 > button > span.MuiButton-label")
    login_btn.click()
    return(driver)

def openStudy(study_path, driver, pgconst):
    '''
    INPUT: path to workspace for study of interest
    
    OUTPUT: None, simply operates in test browser
    
    Notes: login must be run before running this 
    '''
    
    try:
        sbdl_wksp = driver.find_element_by_css_selector(study_path)
        sbdl_wksp.click()
    except:
        time.sleep(3)
        sbdl_wksp = driver.find_element_by_css_selector(study_path)
        sbdl_wksp.click()

    time.sleep(1)
    sbdl_study = driver.find_element_by_css_selector("#root > div > main > div > div.MuiGrid-root.MuiGrid-container.MuiGrid-spacing-xs-4 > div:nth-child(2) > a > div > div > div.MuiGrid-root.MuiGrid-item.MuiGrid-grid-xs-8")
    sbdl_study.click()

    time.sleep(1)
    sbdl_view = driver.find_element_by_css_selector("#root > div > main > div > div > div > div > table > tbody > tr > td:nth-child(4) > a > button > span.MuiButton-label")
    sbdl_view.click()

    time.sleep(5)
    part_btn = driver.find_element_by_css_selector("#root > div > main > div > div > header > div > div > div > div > div > button:nth-child(6)")
    part_btn.click()

    time.sleep(7)

    expand_btn = driver.find_element_by_xpath("/html/body/div[1]/div/main/div/div/div/div/div[2]/div/div/div/table/tfoot/tr/td/div/div[2]/div")
    expand_btn.click()

    sel300 = driver.find_element_by_xpath("/html/body/div[" + str(pgconst) + "]/div[3]/ul/li[3]")
    sel300.click()
    return(driver)

In [36]:
def get_emails(date, driver):
    '''
    INPUT: date of enrollment that you are interested in grabbing
    
    OUTPUT: result list of email and number of submissions
    '''
    table = driver.find_element_by_css_selector('#root > div > main > div > div > div > div > div.MuiPaper-root.MuiCard-root.MuiPaper-elevation1.MuiPaper-rounded > div > div > div > div.jss97 > div > div > div > table > tbody')

#iterate through participants

    participant_email = []
    participant_subm = []
    for row in table.find_elements_by_css_selector('tr'):
        try:
            row_num = row.get_attribute("index")
            try:
                enroll_date = driver.find_element_by_css_selector('#root > div > main > div > div > div > div > div.MuiPaper-root.MuiCard-root.MuiPaper-elevation1.MuiPaper-rounded > div > div > div > div.jss97 > div > div > div > table > tbody > tr:nth-child(' + str(row_num) + ') > td:nth-child(10)')
            except:
                time.sleep(3)
                enroll_date = driver.find_element_by_css_selector('#root > div > main > div > div > div > div > div.MuiPaper-root.MuiCard-root.MuiPaper-elevation1.MuiPaper-rounded > div > div > div > div.jss97 > div > div > div > table > tbody > tr:nth-child(' + str(row_num) + ') > td:nth-child(10)')
            
            if enroll_date.text[:10] == date:
                email_ob = driver.find_element_by_css_selector('#root > div > main > div > div > div > div > div.MuiPaper-root.MuiCard-root.MuiPaper-elevation1.MuiPaper-rounded > div > div > div > div.jss97 > div > div > div > table > tbody > tr:nth-child(' + str(row_num) + ') > td:nth-child(5)')
                email = email_ob.text.split('@')[0]
                print(email)
                participant_email.append(email)
        
                launch_btn = driver.find_element_by_css_selector('#root > div > main > div > div > div > div > div.MuiPaper-root.MuiCard-root.MuiPaper-elevation1.MuiPaper-rounded > div > div > div > div.jss97 > div > div > div > table > tbody > tr:nth-child(' + str(row_num) + ') > td:nth-child(7) > a')
                participant_link = launch_btn.get_attribute('href')
        
                driver.execute_script("window.open('{}');".format(participant_link))
                driver.switch_to.window(driver.window_handles[1])
                time.sleep(3)
                try:
                    fullsched_btn = driver.find_element_by_css_selector('#root > div > main > div > div > div.MuiGrid-root.MuiGrid-item.MuiGrid-grid-xs-8 > div > div > div > div > div > div > div:nth-child(1) > div > div:nth-child(2) > div > button')
                    fullsched_btn.click()
                except:
                    time.sleep(3)
                    fullsched_btn = driver.find_element_by_css_selector('#root > div > main > div > div > div.MuiGrid-root.MuiGrid-item.MuiGrid-grid-xs-8 > div > div > div > div > div > div > div:nth-child(1) > div > div:nth-child(2) > div > button')
                    fullsched_btn.click()
                time.sleep(2)
                submissions = driver.find_element_by_css_selector('#root > div > main > div > div > div.MuiGrid-root.MuiGrid-item.MuiGrid-grid-xs-8 > div > div > div > div > div > div > div.MuiCardContent-root.text-xs-center > div > div:nth-child(4) > h3').text
                print(submissions)
                participant_subm.append(submissions)
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
        except:
            pass
    return(participant_email, participant_subm)
        
       

In [39]:
def full_email_path(study_path, date, pgconst, cutoff):
    '''
    INPUT: study_path, date of enrollment
    
    OUTPUT: result df
    
    Runs everything above in sequence
    '''
    driver = loginMetricWire()
    driver =openStudy(study_path, driver, pgconst)
    participant_email, participant_subm = get_emails(date, driver)
    
    results = pd.DataFrame({
    
        'email': participant_email,
        'surveys': participant_subm,
        'credit': [1 if int(i) > cutoff else 0 for i in participant_subm],
        'action': [float('nan') for i in participant_subm]
    
        })
    return results

In [ ]:
edl_0111 = full_email_path(path_EDL, date, pgconst_EDL, cutoff = 32)

In [47]:
edl_0111.to_csv('./edlIDs.csv')